In [ ]:
# This script is used for interacting with OpenLCA.
# Codes are adapted from Julian Rickert @GreenDelta by Ao Chen @ETH.
# Connect to IPC server in OpenLCA software before running. (Port: 8080)
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('notebook', {'highlight_selected_word': {
    'highlight_across_all_cells': True,
    'only_cells_in_scroll': False,
    'delay': 500,
    'code_cells_only': True,
}})


import pandas as pd
import olca
import uuid
import math
from datetime import datetime

from matplotlib import pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import rcParams
import matplotlib.patches as mpatches
import seaborn as sns

params = {'mathtext.default': 'regular' }

client = olca.Client(8080)
client

In [ ]:
# Get all process dataset in database
process_descriptor = client.get_descriptors(olca.Process)
process_list = []
id_list = []

for process in process_descriptor:
    process_list.append(process.name)
    id_list.append(process.id)
processes_df = pd.DataFrame(list(zip(process_list,
                                   id_list)),
                               columns=['name', 'id'])
processes_df

In [ ]:
# Search specific process databases with key word(s)
# Note that the search is blankspace-sensitive
search_process_df = processes_df[processes_df['name'].str.contains(r'high voltage, production mix(?!$)')] # TODO: change keywords
search_process_df.reset_index(drop=True, inplace=True)

search_process_df

In [ ]:
# Get all flow dataset in database
flow_descriptor = client.get_descriptors(olca.Flow)
flow_list = []
id_list = []

for flow in flow_descriptor:
    flow_list.append(flow.name)
    id_list.append(flow.id)
flows_df = pd.DataFrame(list(zip(flow_list,
                                   id_list)),
                               columns=['name', 'id'])
# flows_df

In [ ]:
# Search specific flow databases with key word(s)
# Note that the search is blankspace-sensitive
search_flow_df = flows_df[flows_df['name'].str.contains(r'oil(?!$)')] # TODO: change keywords
search_flow_df.reset_index(drop=True, inplace=True)

# search_flow_df

In [ ]:
# Create product systems for selected processes
search_list_processID = list(search_process_df['id'])
product_system_ids = []
product_system_names = []
for i in range(len(search_list_processID)):
    product_system = client.create_product_system(search_list_processID[i], default_providers='prefer', preferred_type='UNIT_PROCESS')
    product_system_ids.append(product_system.id)
    ps = client.find(olca.ProductSystem, search_process_df['name'][i])
    product_system_names.append(ps.name)

product_system_df = pd.DataFrame(list(zip(product_system_names, product_system_ids)), 
                    columns=['product system name', 'product system id'])
product_system_df   

In [ ]:
setup = olca.CalculationSetup()
setup.calculation_type = olca.CalculationType.CONTRIBUTION_ANALYSIS
setup.impact_method = client.find(olca.ImpactMethod, 'IPCC 2013')

# amount is the amount of the functional unit (fu) of the system that
# should be used in the calculation; unit, flow property, etc. of the fu
# can be also defined; by default openLCA will take the settings of the
# reference flow of the product system
setup.amount = 1.0

In [ ]:
setup

In [ ]:
for ps in range(len(product_system_names)):
    setup.product_system = client.find(olca.ProductSystem, product_system_names[ps])

    calc_result = client.calculate(setup)

    client.excel_export(calc_result, 'calc_result.xlsx')

    print('Remaining number of processes to be calculated: '+str(ps))

In [ ]:
import pandas as pd

filename_xlsx = 'calc_result.xlsx'
result_db = pd.ExcelFile(filename_xlsx)
impact_contributions_df = result_db.parse('Process impact contributions')

In [ ]:
impact_contributions_df.iloc[2]